<a href="https://colab.research.google.com/github/jaimovi/Analise-de-similaridade-de-Banco-de-Perguntas-por-metodos-NLP/blob/main/Perguntas_ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perguntas
##A ideia deste notebook estruturar inicialmente todas as etapas de pre processamento necessárias e que deverão ser aplicadas, independente de qual modelo . Estabelecer as etapas de transformação em caixa baixa, retirar stopwords, caracteres especiais e etc.



Carrego as Bibliotecas

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
# Load libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load Data
df=pd.read_excel('3 colunas de PERGUNTA.xlsx', usecols = "A,C:C")
df= df.rename(columns={'Texto da Pergunta': 'Pergunta'})
df= df.rename(columns={'Codigo da Pergunta': 'Codigo'})
df= df[:3000]

cabeçalho

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# Simplificar o df para uns 10k~5k



In [ ]:
df.head(100000)

In [ ]:
df.tail(1)

In [ ]:
df.info()

# Exploratory Data Analysis

In [ ]:
print(df.Pergunta)

##Nossa conhecida WordCloud

In [ ]:
# word cloud
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from wordcloud import WordCloud
import re

def clean_text(s):
    if type(s)!=str:
     return ""
    s = re.sub(r'http\S+', '', s)
    s = re.sub('(RT|via)((?:\\b\\W*@\\w+)+)', ' ', s)
    s = re.sub(r'@\S+', '', s)
    s = re.sub('&amp', ' ', s)
    s = re.sub('DO', ' ', s)
    s = re.sub(r'À', '', s)
    return s
df['Pergunta'] = df['Pergunta'].apply(clean_text)

caixa_baixa = df['Pergunta'].to_string().lower()   
wordcloud = WordCloud(
    collocations=False,
    relative_scaling=0.5,
    stopwords=set(stopwords.words('portuguese'))).generate(caixa_baixa)

plt.figure(figsize=(12,12))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
auxiliar = df['Pergunta'].str.lower()
df['Pergunta']= auxiliar
df.head(1000)

In [ ]:
df.dropna(inplace=True)

In [ ]:
import unicodedata

def remove_accents(input_str):
    try:
      nfkd_form = unicodedata.normalize('NFKD', input_str)
    except:
      print(input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return str(only_ascii)

auxiliar = [remove_accents(sentence).replace('b\'','') for sentence in df['Pergunta']]
auxiliar2 = [sentence.replace('\'','') for sentence in auxiliar]
df['Pergunta']= auxiliar2

In [ ]:
df.head()

In [ ]:
todas_stopwords = stopwords.words('portuguese')
todas_stopwords.append('para')

In [ ]:
from nltk.corpus import stopwords
# inserir nova stopword
# stopwords.append('existe')
print(stopwords.words('portuguese'))

In [ ]:
# PROCURAR TEXTO PARA REMOVER STOPWORDS





In [ ]:
# df = df.explode("Pergunta", ignore_index=True)
# df.rename(columns={"Codigo": "Pergunta"}, inplace=True)
# df.index.name = "Code_ID"
# df.head()
# df.to_csv("Perguntas Tokenized.csv")


#Para o caso de bag of words:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# https://stackoverflow.com/questions/8897593/how-to-compute-the-similarity-between-two-text-documents
vect = TfidfVectorizer(min_df=1, stop_words=todas_stopwords)
tfidf = vect.fit_transform(df['Pergunta'][:3000])
pairwise_similarity = tfidf * tfidf.T

In [ ]:
pairwise_similarity

In [ ]:
pd.DataFrame(pairwise_similarity.toarray())

In [ ]:
resultado = pd.DataFrame(pairwise_similarity.toarray())

In [ ]:
import numpy as np
from scipy import stats
   

  
  
print (resultado)
  
print ("\nclipped arr1 : \n", stats.threshold(
        resultado, threshmin = 0.7 , threshmax = 1, newval = 0.85))

In [ ]:
resultado.head()

In [ ]:
resultado.to_excel('resultado.xlsx')

In [ ]:
df['Pergunta'][86], df['Pergunta'][514],df['Pergunta'][133], df['Pergunta'][112]

In [ ]:
# Definir um valor de threshold para classificar como semelhante


# Remover as perguntas semelhantes do conjunto


In [ ]:

resultado[(resultado > 0.99)].any(1)

In [ ]:
to_drop = []
par_perguntas =[]

for column in resultado.columns:
 for index in resultado.index:
  if resultado.loc[index, column] > 0.95 and index != column:
   to_drop.append([index,column])
   par_perguntas.append([df['Pergunta'][index],df['Pergunta'][column]])

In [ ]:
len(to_drop)

In [ ]:
print(to_drop)

In [ ]:
to_drop

In [ ]:
par_perguntas[30]

In [ ]:
par_perguntas

In [ ]:

df['Pergunta'][514],df['Pergunta'][86]

# Aplicação de modelo transformer para similaridade


https://huggingface.co/neuralmind/bert-base-portuguese-cased

https://github.com/neuralmind-ai/portuguese-bert

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
  
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

model = AutoModelForMaskedLM.from_pretrained("neuralmind/bert-base-portuguese-cased")

# ======= SEM UTILIDADE =======

In [ ]:
def get_corpus(caixa_baixa):
    words = []
    for i in caixa_baixa:
        for j in i.split():
            words.append(j.strip())
    return words
corpus = get_corpus(df.Pergunta)
corpus[:100]

In [ ]:
from collections import Counter
counter = Counter(corpus)
most_common = counter.most_common(15)
most_common = dict(most_common)
most_common

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
def get_top_text_ngrams(corpus, n, g):
    vec = CountVectorizer(ngram_range=(g, g)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
plt.figure(figsize = (16,9))
most_common_uni = get_top_text_ngrams(df.clean,10,1)
most_common_uni = dict(most_common_uni)
sns.barplot(x=list(most_common_uni.values()),y=list(most_common_uni.keys()))

In [ ]:
plt.figure(figsize = (16,9))
most_common_bi = get_top_text_ngrams(df.clean,10,2)
most_common_bi = dict(most_common_bi)
sns.barplot(x=list(most_common_bi.values()),y=list(most_common_bi.keys()))

In [ ]:
plt.figure(figsize = (16,9))
most_common_tri = get_top_text_ngrams(df.clean,10,7)
most_common_tri = dict(most_common_tri)
sns.barplot(x=list(most_common_tri.values()),y=list(most_common_tri.keys()))